In [1]:
import numpy as np
import matplotlib.pyplot as plt

from sklearn.externals import joblib

In [2]:
from PIL import Image
import glob
image_list = []
for filename in glob.glob('*.jpg'):
    im = Image.open(filename)
    image_list.append(im)

In [3]:
image_list

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=448x298 at 0x135A75669E8>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=314x209 at 0x135A76092E8>]

In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

In [5]:
for filename in glob.iglob('*.jpg'):
    im = Image.open(filename)
    imResize = im.resize((28,28), Image.ANTIALIAS)
    imResize.save(filename , 'JPEG', quality=90)

In [6]:
image_list = []
for filename in glob.glob('*.jpg'):
    im = Image.open(filename)
    image_list.append(im)
    
image_list

[<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=28x28 at 0x135A8216F28>,
 <PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=28x28 at 0x135A8216FD0>]

In [7]:


img = np.ravel(base)

df = pd.DataFrame([img])
df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,511,512,513,514,515,516,517,518,519,520,521,522,523,524,525,526,527,528,529,530,531,532,533,534,535,536,537,538,539,540,541,542,543,544,545,546,547,548,549,550,551,552,553,554,555,556,557,558,559,560,561,562,563,564,565,566,567,568,569,570,571,572,573,574,575,576,577,578,579,580,581,582,583,584,585,586,587,588,589,590,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605,606,607,608,609,610,611,612,613,614,615,616,617,618,619,620,621,622,623,624,625,626,627,628,629,630,631,632,633,634,635,636,637,638,639,640,641,642,643,644,645,646,647,648,649,650,651,652,653,654,655,656,657,658,659,660,661,662,663,664,665,666,667,668,669,670,671,672,673,674,675,676,677,678,679,680,681,682,683,684,685,686,687,688,689,690,691,692,693,694,695,696,697,698,699,700,701,702,703,704,705,706,707,708,709,710,711,712,713,714,715,716,717,718,719,720,721,722,723,724,725,726,727,728,729,730,731,732,733,734,735,736,737,738,739,740,741,742,743,744,745,746,747,748,749,750,751,752,753,754,755,756,757,758,759,760,761,762,763,764,765,766,767,768,769,770,771,772,773,774,775,776,777,778,779,780,781,782,783,784,785,786,787,788,789,790,791,792,793,794,795,796,797,798,799,800,801,802,803,804,805,806,807,808,809,810,811,812,813,814,815,816,817,818,819,820,821,822,823,824,825,826,827,828,829,830,831,832,833,834,835,836,837,838,839,840,841,842,843,844,845,846,847,848,849,850,851,852,853,854,855,856,857,858,859,860,861,862,863,864,865,866,867,868,869,870,871,872,873,874,875,876,877,878,879,880,881,882,883,884,885,886,887,888,889,890,891,892,893,894,895,896,897,898,899,900,901,902,903,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943,944,945,946,947,948,949,950,951,952,953,954,955,956,957,958,959,960,961,962,963,964,965,966,967,968,969,970,971,972,973,974,975,976,977,978,979,980,981,982,983,984,985,986,987,988,989,990,991,992,993,994,995,996,997,998,999,1000,1001,1002,1003,1004,1005,1006,1007,1008,1009,1010,1011,1012,1013,1014,1015,1016,1017,1018,1019,1020,1021

In [13]:

colourImg = Image.open("DSC05423.jpg")
colourPixels = colourImg.convert("RGB")
colourArray = np.array(colourPixels.getdata())

df = pd.DataFrame(colourArray, columns=["red","green","blue"])
df

,red,green,blue
0,87,120,63
1,73,105,55
2,81,107,68
3,61,80,50
4,102,118,89
5,103,131,91
6,97,144,90
7,65,120,55
8,54,98,35
9,105,128,76


In [14]:
df.index = df.index + 1
df_out = df.stack()
df_out.index = df_out.index.map('{0[1]}_{0[0]}'.format)
df_out.to_frame().T

,red_1,green_1,blue_1,red_2,green_2,blue_2,red_3,green_3,blue_3,red_4,green_4,blue_4,red_5,green_5,blue_5,red_6,green_6,blue_6,red_7,green_7,blue_7,red_8,green_8,blue_8,red_9,green_9,blue_9,red_10,green_10,blue_10,red_11,green_11,blue_11,red_12,green_12,blue_12,red_13,green_13,blue_13,red_14,green_14,blue_14,red_15,green_15,blue_15,red_16,green_16,blue_16,red_17,green_17,blue_17,red_18,green_18,blue_18,red_19,green_19,blue_19,red_20,green_20,blue_20,red_21,green_21,blue_21,red_22,green_22,blue_22,red_23,green_23,blue_23,red_24,green_24,blue_24,red_25,green_25,blue_25,red_26,green_26,blue_26,red_27,green_27,blue_27,red_28,green_28,blue_28,red_29,green_29,blue_29,red_30,green_30,blue_30,red_31,green_31,blue_31,red_32,green_32,blue_32,red_33,green_33,blue_33,red_34,green_34,blue_34,red_35,green_35,blue_35,red_36,green_36,blue_36,red_37,green_37,blue_37,red_38,green_38,blue_38,red_39,green_39,blue_39,red_40,green_40,blue_40,red_41,green_41,blue_41,red_42,green_42,blue_42,red_43,green_43,blue_43,red_44,green_44,blue_44,red_45,green_45,blue_45,red_46,green_46,blue_46,red_47,green_47,blue_47,red_48,green_48,blue_48,red_49,green_49,blue_49,red_50,green_50,blue_50,red_51,green_51,blue_51,red_52,green_52,blue_52,red_53,green_53,blue_53,red_54,green_54,blue_54,red_55,green_55,blue_55,red_56,green_56,blue_56,red_57,green_57,blue_57,red_58,green_58,blue_58,red_59,green_59,blue_59,red_60,green_60,blue_60,red_61,green_61,blue_61,red_62,green_62,blue_62,red_63,green_63,blue_63,red_64,green_64,blue_64,red_65,green_65,blue_65,red_66,green_66,blue_66,red_67,green_67,blue_67,red_68,green_68,blue_68,red_69,green_69,blue_69,red_70,green_70,blue_70,red_71,green_71,blue_71,red_72,green_72,blue_72,red_73,green_73,blue_73,red_74,green_74,blue_74,red_75,green_75,blue_75,red_76,green_76,blue_76,red_77,green_77,blue_77,red_78,green_78,blue_78,red_79,green_79,blue_79,red_80,green_80,blue_80,red_81,green_81,blue_81,red_82,green_82,blue_82,red_83,green_83,blue_83,red_84,green_84,blue_84,red_85,green_85,blue_85,red_86,green_86,blue_86,red_87,green_87,blue_87,red_88,green_88,blue_88,red_89,green_89,blue_89,red_90,green_90,blue_90,red_91,green_91,blue_91,red_92,green_92,blue_92,red_93,green_93,blue_93,red_94,green_94,blue_94,red_95,green_95,blue_95,red_96,green_96,blue_96,red_97,green_97,blue_97,red_98,green_98,blue_98,red_99,green_99,blue_99,red_100,green_100,blue_100,red_101,green_101,blue_101,red_102,green_102,blue_102,red_103,green_103,blue_103,red_104,green_104,blue_104,red_105,green_105,blue_105,red_106,green_106,blue_106,red_107,green_107,blue_107,red_108,green_108,blue_108,red_109,green_109,blue_109,red_110,green_110,blue_110,red_111,green_111,blue_111,red_112,green_112,blue_112,red_113,green_113,blue_113,red_114,green_114,blue_114,red_115,green_115,blue_115,red_116,green_116,blue_116,red_117,green_117,blue_117,red_118,green_118,blue_118,red_119,green_119,blue_119,red_120,green_120,blue_120,red_121,green_121,blue_121,red_122,green_122,blue_122,red_123,green_123,blue_123,red_124,green_124,blue_124,red_125,green_125,blue_125,red_126,green_126,blue_126,red_127,green_127,blue_127,red_128,green_128,blue_128,red_129,green_129,blue_129,red_130,green_130,blue_130,red_131,green_131,blue_131,red_132,green_132,blue_132,red_133,green_133,blue_133,red_134,green_134,blue_134,red_135,green_135,blue_135,red_136,green_136,blue_136,red_137,green_137,blue_137,red_138,green_138,blue_138,red_139,green_139,blue_139,red_140,green_140,blue_140,red_141,green_141,blue_141,red_142,green_142,blue_142,red_143,green_143,blue_143,red_144,green_144,blue_144,red_145,green_145,blue_145,red_146,green_146,blue_146,red_147,green_147,blue_147,red_148,green_148,blue_148,red_149,green_149,blue_149,red_150,green_150,blue_150,red_151,green_151,blue_151,red_152,green_152,blue_152,red_153,green_153,blue_153,red_154,green_154,blue_154,red_155,green_155,blue_155,red_156,green_156,blue_156,red_157,green_157,blue_157,red_158,green_158,blue_158,red_159,green_159,blue_159,red_160,green_160,blue_160,red

In [15]:
colourImg = Image.open("DSC05423.jpg")

In [44]:
import mahotas
# feature-descriptor-1: Hu Moments
def fd_hu_moments(image):
    image = np.asarray(image)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    feature = cv2.HuMoments(cv2.moments(image)).flatten()
    return feature

def fd_haralick(image):
    image = np.asarray(image)
    # convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    # compute the haralick texture feature vector
    haralick = mahotas.features.haralick(gray).mean(axis=0)
    # return the result
    return haralick

In [49]:

fd_haralick(base)

array([ 1.03850838e-03,  3.63773491e+02,  6.36218017e-01,  4.98848124e+02,
        8.79985757e-02,  2.51847884e+02,  1.63161900e+03,  7.06754619e+00,
        1.00776055e+01,  1.69415515e-04,  5.10921935e+00, -4.11772293e-01,
        9.97295628e-01])

In [46]:
aa = fd_hu_moments(base)
aaa = fd_haralick(base)


In [47]:
global_feature = np.hstack([aa, aaa])

In [48]:
global_feature

array([ 1.30037712e-03,  4.53190298e-10,  4.47156606e-12,  6.59240246e-13,
       -2.09966480e-25,  7.70240167e-18, -1.11222163e-24,  1.03850838e-03,
        3.63773491e+02,  6.36218017e-01,  4.98848124e+02,  8.79985757e-02,
        2.51847884e+02,  1.63161900e+03,  7.06754619e+00,  1.00776055e+01,
        1.69415515e-04,  5.10921935e+00, -4.11772293e-01,  9.97295628e-01])

In [50]:
type(global_feature)

numpy.ndarray

In [59]:
def fd_histogram(image, mask=None):
    image = np.asarray(image)
    # convert the image to HSV color-space
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    # compute the color histogram
    hist  = cv2.calcHist([image], [0, 1, 2], None, [5,5,5],[0, 256, 0, 256, 0, 256])
    # normalize the histogram00
    cv2.normalize(hist, hist)
    # return the histogram
    return hist.flatten()

In [60]:
fd_histogram(base, mask=None)

array([0.        , 0.00271513, 0.01357563, 0.04615715, 0.        ,
       0.        , 0.        , 0.18191348, 0.02443614, 0.        ,
       0.        , 0.        , 0.03258152, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04344203, 0.1737681 , 0.12489583, 0.        ,
       0.        , 0.01357563, 0.8661254 , 0.39097822, 0.        ,
       0.        , 0.00543025, 0.02443614, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04615715, 0.0923143 , 0.01629076, 0.        ,
       0.        , 0.01900589, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

In [61]:
aa = fd_hu_moments(base)
aaa = fd_haralick(base)
aaaa = fd_histogram(base, mask=None)

In [62]:
global_feature = np.hstack([aa, aaa, aaaa])
global_feature

array([ 1.30037712e-03,  4.53190298e-10,  4.47156606e-12,  6.59240246e-13,
       -2.09966480e-25,  7.70240167e-18, -1.11222163e-24,  1.03850838e-03,
        3.63773491e+02,  6.36218017e-01,  4.98848124e+02,  8.79985757e-02,
        2.51847884e+02,  1.63161900e+03,  7.06754619e+00,  1.00776055e+01,
        1.69415515e-04,  5.10921935e+00, -4.11772293e-01,  9.97295628e-01,
        0.00000000e+00,  2.71512661e-03,  1.35756331e-02,  4.61571515e-02,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  1.81913480e-01,
        2.44361386e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        3.25815193e-02,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  4.34420258e-02,  1.73768103e-01,
        1.24895826e-01,  0.00000000e+00,  0.00000000e+00,  1.35756331e-02,
        8.66125405e-01,  

In [65]:

df = pd.DataFrame(global_feature.flatten())


In [66]:
df

,0
0,1.300377e-03
1,4.531903e-10
2,4.471566e-12
3,6.592402e-13
4,-2.099665e-25
5,7.702402e-18
6,-1.112222e-24
7,1.038508e-03
8,3.637735e+02
9,6.362180e-01


In [67]:
df.index = df.index + 1
df_out = df.stack()
df_out.index = df_out.index.map('{0[1]}_{0[0]}'.format)
df_out.to_frame().T

,0_1,0_2,0_3,0_4,0_5,0_6,0_7,0_8,0_9,0_10,0_11,0_12,0_13,0_14,0_15,0_16,0_17,0_18,0_19,0_20,0_21,0_22,0_23,0_24,0_25,0_26,0_27,0_28,0_29,0_30,0_31,0_32,0_33,0_34,0_35,0_36,0_37,0_38,0_39,0_40,0_41,0_42,0_43,0_44,0_45,0_46,0_47,0_48,0_49,0_50,0_51,0_52,0_53,0_54,0_55,0_56,0_57,0_58,0_59,0_60,0_61,0_62,0_63,0_64,0_65,0_66,0_67,0_68,0_69,0_70,0_71,0_72,0_73,0_74,0_75,0_76,0_77,0_78,0_79,0_80,0_81,0_82,0_83,0_84,0_85,0_86,0_87,0_88,0_89,0_90,0_91,0_92,0_93,0_94,0_95,0_96,0_97,0_98,0_99,0_100,0_101,0_102,0_103,0_104,0_105,0_106,0_107,0_108,0_109,0_110,0_111,0_112,0_113,0_114,0_115,0_116,0_117,0_118,0_119,0_120,0_121,0_122,0_123,0_124,0_125,0_126,0_127,0_128,0_129,0_130,0_131,0_132,0_133,0_134,0_135,0_136,0_137,0_138,0_139,0_140,0_141,0_142,0_143,0_144,0_145
0,0.0013,4.531903e-10,4.471566e-12,6.592402e-13,-2.099665e-25,7.702402e-18,-1.112222e-24,0.001039,363.773491,0.636218,498.848124,0.087999,251.847884,1631.619003,7.067546,10.077605,0.000169,5.109219,-0.411772,0.997296,0.0,0.002715,0.013576,0.046157,0.0,0.0,0.0,0.181913,0.024436,0.0,0.0,0.0,0.032582,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043442,0.173768,0.124896,0.0,0.0,0.013576,0.866125,0.390978,0.0,0.0,0.00543,0.024436,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.046157,0.092314,0.016291,0.0,0.0,0.019006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.010861,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
